In [29]:
import numpy as np

In [30]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [31]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()             # 함수를 가져온다.
            x, y = f.input, f.output    # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad) # backward 메서드를 호출한다.

            if x.creator is not None:
                funcs.append(x.creator)

In [32]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]  # 데이터를 꺼낸다.
        ys = self.forward(xs)          # 구체적인 계산은 forward 메서드에서 한다.

        outputs = [Variable(as_array(y)) for y in ys]  # Variable 형태로 되돌린다.
        for output in outputs:
            output.set_creator(self) # 출력 변수에 창조자를 설정한다.

        self.inputs  = inputs       # 입력 변수를 기억(보관)한다.
        self.outputs = outputs      # 출력도 저장한다.
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [33]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y, )

In [34]:
xs = [Variable(np.array(2)), Variable(np.array(3))] # 리스트로 준비
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5
